In [1]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import os
import json
import time
import requests
import math as m
import pandas as pd
import numpy as np
import koios_python as kp



# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 1000

## NoteBook 1


#### In this Notebook we will be going over how to use the koios_python library in various scenarios a developer may face. The first thing we need to do is create a class instance for koios_python and choose what network we would like to use for our queries, in this tutorial we will use mainnet but the library supports all the various developer networks available including preview, preprod, guild, and mainnet.

In [4]:
# create a koios server object with network set to mainnet

koios_mainnet = kp.URLs(network='mainnet')

# let's check the current tip of the mainnet
koios_mainnet.get_tip()

### Let's go over some use case examples for koios_python

NFTs and other Assets:

What if you wanted to find out how many NFTs are on the cardano blockchain?

How many wallets hold a specific asset or token?

How many pools are there?

How much ada is going to the treasury per epoch?
